### **Q_1**

In [0]:
import os
print(os.getcwd())  # Check the current working directory

### **Q_2**

In [0]:
import os
print(os.path.exists("/Workspace/Users/a845678@asb.dtcbtndsie.onmicrosoft.com/assignment/resources/config.yml"))  # Check if the file exists
print(os.path.abspath("/Workspace/Users/a845678@asb.dtcbtndsie.onmicrosoft.com/assignment/resources/config.yml"))  # Print the absolute path


### **Q_3**